# Comparison between patients and controls

In this notebook we made the analysis for matched data according to `sex` and `age_at_onset` variables. The summary statistics compare patients and controls. 

In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)
matched_ids = pd.read_csv(
    os.path.join(folder_path, 'matched_ids.csv'), 
    low_memory=False
)
df = df.merge(matched_ids, on='participant_id')

## Summary statistics for demography and lifestyle

In [ ]:
# rename and select first group of variables

variables_dict_1 = {
    "age"                       : "Age at Recruitment (years)",
    "age_at_onset"              : "Age at Onset (years)" ,
    "bmi_comp"                  : "Body Mass Index",
    "PD_duration"               : "Duration of Disease (years)",
    "years_of_education"        : "Years of Education",
    "agecat_1"                  : "Onset <50 years" ,
    "agecat_2"                  : "Onset <40 years",
    "work"                      : "Currently Employed",
    "reason"                    : "PD-related Reason for Stop Working",
    "nature_of_work___1"        : "Agriculture as a job",
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "during_your_lifetime_did_y": "Exposure to chemicals to kill insects in your life time",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",
    "have_you_ever_had_a_head_i": "Head injury or concussion",
}
variables_1 = ['participant_id', 'Status'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Onset <50 years", "Onset <40 years", "Currently Employed", "PD-related Reason for Stop Working", 
               "Agriculture as a job", "Life time direct exposure to pesticide/insecticide/fungicide", 
               "Exposure to chemicals to kill insects in your life time", "Smoked 100 or more cigarettes in lifetime", 
               "Regular consumption of caffeinated drinks for >6months", "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

groups = ['Patient', 'Control']

In [ ]:
# keep only variables for analysis

df_1 = df[variables_1].copy()
df_1 = df_1.rename(columns=variables_dict_1)

In [ ]:
mask_control = (df_1['Status']=='Control')

df_1.loc[mask_control, "Age at Onset (years)"] = df_1.loc[mask_control, "Age at Recruitment (years)"]

In [ ]:
# recode variable values

recode_dict = {
    "Currently Employed"                :{'Yes':1, 'No':0},
    "Agriculture as a job"              :{'Checked':1, 'Unchecked':0},
    "Onset <50 years"                   :{'<50':1, '>=50':0},
    "Onset <40 years"                   :{'<40':1, '>=40':0},
    "PD-related Reason for Stop Working":{'PD related':1, 'not PD related':0}
}
for key in recode_dict.keys():
    df_1 = categories_recoder(df_1, [key], recode_dict[key])

recode = ["Life time direct exposure to pesticide/insecticide/fungicide", 
          "Exposure to chemicals to kill insects in your life time", "Smoked 100 or more cigarettes in lifetime",
          "Regular consumption of caffeinated drinks for >6months", "Head injury or concussion"]
mapping = {'Yes':1, 'No':0, 'Dont Know':np.nan, 'Refused':np.nan, 'Possibly':np.nan}

df_1 = categories_recoder(df_1, recode, mapping)

In [ ]:
summary = [
    two.report_mean_std(
        data_df    =df_1,
        variables  =stats_meas['mean'],
        groups     =groups,
        grouping_by='Status'
    ),
    two.report_median_iqr(
        data_df    =df_1,
        variables  =stats_meas['median'],
        groups     =groups,
        grouping_by='Status'
    ),
    two.report_proportion(
        data_df    =df_1,
        variables  =stats_meas['n'],
        groups     =groups,
        grouping_by='Status'
    )
]

# Analysis of patients medical history

In [ ]:
# rename and select second group of variables

variables_dict_2 = {
    "medical_history_neurologic___7": "Stroke",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___3" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_psychologi___1": "History of Anxiety",
    "medical_history_psychologi___3": "History of Depression",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor",
    "family_member_diagnosed"       : "Family History of Dementia"
}
variables_2 = ['participant_id', 'Status'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "History of Depression", "Family History of PD", "Family History of Tremor", "Family History of Dementia"]
}

In [ ]:
# keep only variables for analysis

df_2 = df[variables_2].copy()
df_2 = df_2.rename(columns=variables_dict_2)

In [ ]:
# recode variable values

fam_hist = ["Family History of PD", "Family History of Tremor", "Family History of Dementia"]
remaining= [feat for feat in stats_meas2['n'] if feat not in fam_hist]

df_2 = categories_recoder(df_2, fam_hist, {"Yes":1, "No":0})
df_2 = categories_recoder(df_2, remaining, {"Checked":1, "Unchecked":0})

In [ ]:
# count and proportion of categorical variables

df_sum4 = two.report_proportion(
    data_df    =df_2,
    variables  =stats_meas2['n'],
    groups     =groups,
    grouping_by='Status',
    subheader  ='Medical History'
)

In [ ]:
# save result in csv file

df_4 = pd.concat(summary + [df_sum4], axis=0,ignore_index=True)

df_4 = two.final_formatter(df_4, groups)

mask1 = df_4['Variable'].apply( lambda x: "Family" in x)
mask2 = (df_4['Variable']=='PD-related Reason for Stop Working')

df_4.loc[mask1, 'p-value'] = ''
df_4.loc[mask2, 'p-value'] = ''

df_4.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_4.csv'), index=False)
df_4